In [1]:
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/20.2 MB ? eta -:--:--
   ---------------------------------------- 0.2/20.2 MB 3.9 MB/s eta 0:00:06
   - -------------------------------------- 0.5/20.2 MB 5.6 MB/s eta 0:00:04
   - -------------------------------------- 0.9/20.2 MB 6.7 MB/s eta 0:00:03
   -- ------------------------------------- 1.4/20.2 MB 7.3 MB/s eta 0:00:03
   --- ------------------------------------ 1.7/20.2 MB 7.4 MB/s eta 0:00:03
   ---- ----------------------------------- 2.2/20.2 MB 7.7 MB/s eta 0:00:03
   ----- ---------------------------------- 2.5/20.2 MB 8.1 MB/s eta 0:00:03
   ----- ---------------------------------- 2.5/20.2 MB 8.1 MB/s eta 0:00:03
   ------ --------------------------------- 3.2/20.2 MB 7.6 MB/s eta 0:00:03
   ------- ----------------------------

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/pvateekul/2110446_DSDE_2023s2/main/code/Week03_ML/mushroom2020_dataset.csv')

In [4]:
df.isnull().sum()

id                               0
label                           60
cap-shape                        0
cap-surface                     27
bruises                         99
odor                            99
gill-attachment                 99
gill-spacing                   130
gill-size                      121
stalk-shape                    121
stalk-root                      31
stalk-surface-above-ring        31
stalk-surface-below-ring        31
veil-type                       62
ring-number                     62
ring-type                       62
spore-print-color               56
population                      56
habitat                         31
cap-color-rate                  27
gill-color-rate                121
veil-color-rate                 62
stalk-color-above-ring-rate     31
stalk-color-below-ring-rate     62
dtype: int64

Data Preparation

In [5]:
df.dropna(subset=['label'], inplace=True)
df.shape

(5764, 24)

In [6]:
drop_columns = ['id','gill-attachment', 'gill-spacing', 'gill-size','gill-color-rate','stalk-root', 'stalk-surface-above-ring',
 'stalk-surface-below-ring', 'stalk-color-above-ring-rate','stalk-color-below-ring-rate','veil-color-rate','veil-type']

df.drop(columns=drop_columns, inplace=True)
df.shape

(5764, 12)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5764 entries, 0 to 5823
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   label              5764 non-null   object 
 1   cap-shape          5764 non-null   object 
 2   cap-surface        5737 non-null   object 
 3   bruises            5665 non-null   object 
 4   odor               5665 non-null   object 
 5   stalk-shape        5643 non-null   object 
 6   ring-number        5702 non-null   object 
 7   ring-type          5702 non-null   object 
 8   spore-print-color  5708 non-null   object 
 9   population         5708 non-null   object 
 10  habitat            5733 non-null   object 
 11  cap-color-rate     5737 non-null   float64
dtypes: float64(1), object(11)
memory usage: 585.4+ KB


In [8]:
mapping_dict = {
    "label": {
        "p": 0,
        "e": 1,
    },
}

df = df.replace(mapping_dict)
df.label.value_counts()

C:\Users\Admin\AppData\Local\Temp\ipykernel_27024\3373781591.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(mapping_dict)


label
0    3660
1    2104
Name: count, dtype: int64

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

cat_cols=['cap-surface','bruises','odor','stalk-shape','ring-number','ring-type','spore-print-color','population','habitat']
num_cols=['cap-color-rate']

num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', StandardScaler())])

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse_output=False))])
df

,label,cap-shape,cap-surface,bruises,odor,stalk-shape,ring-number,ring-type,spore-print-color,population,habitat,cap-color-rate
0,0,x,s,t,p,e,o,p,k,s,u,1.0
1,1,x,s,t,a,e,o,p,n,n,g,2.0
2,1,b,s,t,l,e,o,p,n,n,m,3.0
3,0,x,y,t,p,e,o,p,k,s,u,3.0
4,1,x,s,f,n,t,o,e,n,a,g,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5819,1,k,s,f,n,e,o,p,b,c,l,1.0
5820,1,x,s,f,n,e,o,p,b,v,l,1.0
5821,1,f,s,f,n,e,o,p,b,c,l,1.0
5822,0,k,y,f,y,t,o,e,w,v,l,1.0


In [10]:
from sklearn.compose import ColumnTransformer

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline', num_pipeline, num_cols),
    ('cat_pipeline', cat_pipeline, cat_cols)],
    n_jobs=-1)

col_trans

ColumnTransformer(n_jobs=-1,
                  transformers=[('num_pipeline',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scale', StandardScaler())]),
                                 ['cap-color-rate']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one-hot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['cap-surface', 'bruises', 'odor',
                                  'stalk-shape', 'ring-number', 'ring-type',
                                  'spore-print-color', 'population',
                                  'habitat'])])

In [11]:
from sklearn.model_selection import train_test_split
y = df.pop('label')
x = df

x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y,test_size=0.2, random_state=2020)
print(x_train.shape)
print(x_test.shape)

(4611, 11)
(1153, 11)


Random Forest

In [12]:
x_train

,cap-shape,cap-surface,bruises,odor,stalk-shape,ring-number,ring-type,spore-print-color,population,habitat,cap-color-rate
3526,f,y,t,n,e,t,e,w,c,w,5.0
1214,f,y,t,n,t,o,p,k,y,d,5.0
1841,f,y,t,n,t,o,p,n,v,d,5.0
5548,k,s,f,f,t,o,e,w,v,d,1.0
3379,x,y,t,n,e,t,e,w,c,w,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2647,x,y,f,f,e,o,l,h,y,d,4.0
5265,k,y,f,y,t,o,e,w,v,d,5.0
5015,x,s,f,n,e,t,p,w,n,g,3.0
356,x,y,t,n,t,o,p,k,y,d,5.0


In [13]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [14]:
from sklearn.ensemble import RandomForestClassifier

local_registry = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(local_registry)
experiment_id = mlflow.set_experiment('test_experiment')

def eval_metrics(actual, pred):
    accuracy = accuracy_score(actual, pred)
    return accuracy

def train_model(criterion, max_depth, min_samples_leaf,n_estimators):

    # Starting the Experiement
    with mlflow.start_run():

        # Model building
        model = RandomForestClassifier(criterion=criterion, max_depth= max_depth, min_samples_leaf = min_samples_leaf,
                                        n_estimators = n_estimators,random_state=2020)
        model_pipeline = Pipeline(steps=[
            ('col_trans', col_trans),
            ('model', model)
        ])
        model_pipeline.fit(x_train, y_train)
        y_pred = model_pipeline.predict(x_test)

        (accuracy) = eval_metrics(y_test, y_pred)

        print('Decirion tree (criterion=%s, max_depth=%d, min_samples_leaf=%d, n_estimators=%d):'%(criterion, max_depth, min_samples_leaf, n_estimators))
        print('Accuracy: {:.4f}'.format(accuracy))

        # Logging Parameters
        mlflow.log_param("criterion", criterion)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("min_samples_leaf", min_samples_leaf)
        mlflow.log_param("n_estimators", n_estimators)

        # Logging Metrics
        mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))

        # Model Logging
        mlflow.sklearn.log_model(model_pipeline, 'model')

        return model_pipeline

2024/04/22 23:03:58 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/04/22 23:03:59 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [15]:
critetion = ['gini','entropy']
max_depth = [2,3,6]
min_samples_leaf = [2,5,10]
n_estimators =[100,200]

for i in range(len(critetion)):
    for j in range(len(max_depth)):
        for k in range(len(min_samples_leaf)):
            for l in range(len(n_estimators)):
                train_model(critetion[i], max_depth[j], min_samples_leaf[k], n_estimators[l])

Decirion tree (criterion=gini, max_depth=2, min_samples_leaf=2, n_estimators=100):
Accuracy: 0.9714
Decirion tree (criterion=gini, max_depth=2, min_samples_leaf=2, n_estimators=200):
Accuracy: 0.9705
Decirion tree (criterion=gini, max_depth=2, min_samples_leaf=5, n_estimators=100):
Accuracy: 0.9714
Decirion tree (criterion=gini, max_depth=2, min_samples_leaf=5, n_estimators=200):
Accuracy: 0.9705
Decirion tree (criterion=gini, max_depth=2, min_samples_leaf=10, n_estimators=100):
Accuracy: 0.9705
Decirion tree (criterion=gini, max_depth=2, min_samples_leaf=10, n_estimators=200):
Accuracy: 0.9696
Decirion tree (criterion=gini, max_depth=3, min_samples_leaf=2, n_estimators=100):
Accuracy: 0.9792
Decirion tree (criterion=gini, max_depth=3, min_samples_leaf=2, n_estimators=200):
Accuracy: 0.9748
Decirion tree (criterion=gini, max_depth=3, min_samples_leaf=5, n_estimators=100):
Accuracy: 0.9792
Decirion tree (criterion=gini, max_depth=3, min_samples_leaf=5, n_estimators=200):
Accuracy: 0.974

In [16]:
best_run_df = mlflow.search_runs(order_by=['metrics.accuracy DESC'], max_results=10)
best_run_df

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.criterion,params.n_estimators,params.max_depth,params.min_samples_leaf,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name
0,5329ca494fed4e47ae9436c1aa247297,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:06:29.884000+00:00,2024-04-22 16:06:34.375000+00:00,0.999133,entropy,200,6,5,"[{""run_id"": ""5329ca494fed4e47ae9436c1aa247297""...",abundant-asp-751,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
1,4c6f986e6453407a9b9948910759f78d,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:06:25.843000+00:00,2024-04-22 16:06:29.867000+00:00,0.999133,entropy,100,6,5,"[{""run_id"": ""4c6f986e6453407a9b9948910759f78d""...",vaunted-fox-659,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
2,7348d130887e41dc8eae7e3e408bccb8,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:06:21.576000+00:00,2024-04-22 16:06:25.828000+00:00,0.999133,entropy,200,6,2,"[{""run_id"": ""7348d130887e41dc8eae7e3e408bccb8""...",smiling-slug-119,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
3,84d30d0b55284b468713271844227f38,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:06:17.299000+00:00,2024-04-22 16:06:21.560000+00:00,0.999133,entropy,100,6,2,"[{""run_id"": ""84d30d0b55284b468713271844227f38""...",industrious-hare-955,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
4,1888838d60614f3b948185897eba615d,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:05:21.128000+00:00,2024-04-22 16:05:25.234000+00:00,0.999133,gini,200,6,10,"[{""run_id"": ""1888838d60614f3b948185897eba615d""...",bold-ox-730,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
5,23ac0363567f4b0fbf7466ddd71caab9,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:05:13.032000+00:00,2024-04-22 16:05:17.121000+00:00,0.999133,gini,200,6,5,"[{""run_id"": ""23ac0363567f4b0fbf7466ddd71caab9""...",dashing-shrew-203,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
6,a228c938c3ff4c618ab6d9103621fc58,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:05:08.614000+00:00,2024-04-22 16:05:13.012000+00:00,0.999133,gini,100,6,5,"[{""run_id"": ""a228c938c3ff4c618ab6d9103621fc58""...",unruly-chimp-35,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
7,e522ad6f14984c94b7742c701c103bf6,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:05:03.748000+00:00,2024-04-22 16:05:08.593000+00:00,0.999133,gini,200,6,2,"[{""run_id"": ""e522ad6f14984c94b7742c701c103bf6""...",gifted-asp-842,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
8,22ab1c7861ac46f08cba0aa682ce5e54,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:04:59.816000+00:00,2024-04-22 16:05:03.733000+00:00,0.999133,gini,100,6,2,"[{""run_id"": ""22ab1c7861ac46f08cba0aa682ce5e54""...",capricious-perch-286,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...
9,528309249aff480d9883cb222063e9be,1,FINISHED,file:///c:/ThisTermProj/Data-Sci-Eng-Final/Wee...,2024-04-22 16:06:38.646000+00:00,2024-04-22 16:06:42.491000+00:00,0.998265,entropy,200,6,10,"[{""run_id"": ""528309249aff480d9883cb222063e9be""...",carefree-ant-445,Admin,LOCAL,c:\Python312\Lib\site-packages\ipykernel_launc...


In [17]:
run_id = str(best_run_df.loc[0, 'run_id'])
print('run_id: ', run_id)

run_id = str(best_run_df.loc[0, 'run_id'])
model_uri = f"runs:/{run_id}/model"
print('model_uri: ', model_uri)

run_id:  5329ca494fed4e47ae9436c1aa247297
model_uri:  runs:/5329ca494fed4e47ae9436c1aa247297/model


In [18]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri=f"runs:/{run_id}/model")

# Predict on a Pandas DataFrame.
predicted = loaded_model.predict(pd.DataFrame(x_test))

print(classification_report(y_test, predicted, target_names=['0', '1'], digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.9986    0.9993       732
           1     0.9976    1.0000    0.9988       421

    accuracy                         0.9991      1153
   macro avg     0.9988    0.9993    0.9991      1153
weighted avg     0.9991    0.9991    0.9991      1153



In [19]:
from sklearn.metrics import classification_report,confusion_matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, predicted,labels = [0,1]))

Confusion Matrix:
[[731   1]
 [  0 421]]
